# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt

import numpy as np
import random

# Reading data

In [3]:
df = pd.read_csv('data/fpl_fbref_elo_players.csv')

In [4]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# New features

In [5]:
# average goals for and against team in last 5 matches
grouped = df.groupby('Name')

In [6]:
df['ScoreForLast5'] = grouped['Team Score'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

In [7]:
df['ScoreAgainstLast5'] = grouped['Opp Score'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

In [8]:
df['ScoreForLast5'] = df['ScoreForLast5'].shift(1)
df['ScoreAgainstLast5'] = df['ScoreAgainstLast5'].shift(1)

In [9]:
df[df["Name"] == "Mohamed-Salah"][["ScoreForLast5", "ScoreAgainstLast5", "Team Score", "Opp Score"]]

,ScoreForLast5,ScoreAgainstLast5,Team Score,Opp Score
0,NaN,NaN,2,2
1,2.000000,2.000000,1,1
2,1.500000,1.500000,1,2
3,1.333333,1.666667,9,0
4,3.250000,1.250000,2,1
5,3.000000,1.200000,0,0
6,2.600000,0.800000,3,3
7,3.000000,1.200000,2,3
8,3.200000,1.400000,1,0
9,1.600000,1.400000,1,0


# Saving df to file

In [10]:
df.to_csv('data/final_dataset.csv')

In [12]:
df.iloc[1]

Date                     2022-08-15
Day                             Mon
Comp                 Premier League
Round                   Matchweek 2
Venue                          Home
Squad                     Liverpool
Opponent             Crystal Palace
Start                             Y
Pos                           RW,FW
Min                            90.0
Gls                             0.0
Ast                             0.0
PK                              0.0
PKatt                           0.0
Sh                              3.0
SoT                             1.0
CrdY                            0.0
CrdR                            0.0
Touches                        51.0
Tkl                             0.0
Int                             0.0
Blocks                          0.0
xG                              0.3
npxG                            0.3
xAG                             0.8
SCA                             9.0
GCA                             0.0
Cmp                         